# Learning Bitmaths for Bitmaps

## Introduction

Bitmaps are used to index data compactly. This can be extremely useful when building blockchain applications in resource constrained environments. A notable use case pioneered by Uniswap V3 is for concentrated liquidity AMMs which need to often and efficiently query the next chunk of available liquidity to swap through.

A bitmap is made up of **words** which are chunks of binary strings. We query each word by passing it's index into a map from. We need to chunk out the map into separate words because because a single integer can only hold a finite number of bits (*note: Python ints are 32 bits long so some details may change depending on the language you're using, but the general logic is still the same.*)

In [769]:
#Initializing the bitmap and 3 words
bitmap = {} 
bitmap[-2] = 0
bitmap[-1] = 0
bitmap[0] = 0
bitmap[1] = 0
bitmap[2] = 0

## Flipping Ticks

### Right Shifts for Floor Division

The first bit trick used in the bitmap is the equivalence of >> x and floor division by 2 ** x. This is because for every bit you shift right, you floor divide the number by 2. So by doing it x times, we get floor division by 2 ** x. 

This is used to figure out which word to check for a given tick since it perfectly segments the global tick indices into the number of words there are i.e., if we the max number of ticks is 100, and each word is 32 bits, then we need 4 words to house all of the tick data (1 -> 0-31, 2 -> 32-63, 3 -> 64-95, 4 -> 96-99). 

In [770]:
assert int(21 / 32) == 21 >> 5 == 0
print(int(21 / 32), 21 >> 5, 0)

assert int(61 / 32) == 61 >> 5 == 1
print(int(61 / 32), 61 >> 5, 2)

assert int(66 / 32) == 66 >> 5 == 2
print(int(66 / 32), 66 >> 5, 2)

0 0 0
1 1 2
2 2 2


### Modulo Division to Retrieve Local Tick Index

We also use modulo division to figure out what the "local index" is of tick i.e., the index within a particular word.

In [771]:
assert 21 % 32 == 21
print(21 % 32, 21)

assert 61 % 32 == 29
print(61 % 32, 29)

assert 66 % 32 == 2
print(66 % 32, 2)

21 21
29 29
2 2


### Grabbing the Local Tick Index and Word Index

Let's put them together to create a function which returns the position.

In [772]:
from typing import Tuple

def position(tickIndex : int) -> Tuple[int, int]:
    # Remember that >> 5 is equivalent to floor division by 2**5
    wordIndex = tickIndex >> 5

    # Used to determine the "local index" of the tick within the word.
    localTickIndex = tickIndex % 32
    return wordIndex, localTickIndex


### Left Shift for "Mask" Generation

The next bit trick used is the creation of a mask that can be used to flip ticks on and off. This trick uses the fact that 1 is represented in binary as "0...001" . Thus by bitshifting it leftward by x bits, we ensure that there is now a 1, x places to the left i.e., index (*note: indexing is left to right in binary*).

This max will be used in bitwise XOR operations in the future to turn ticks on (if they're off and are being initialized - first liquidity is being deposited), or off (if they're on and being unitialized - all liquidity is being withdrawn).

In [773]:
# 1 == "0...001"
binaryString = 1
print("{0:032b}".format(binaryString))

#1 << 5 == "0...0100000"
binaryString = binaryString << 5
print("{0:032b}".format(binaryString))

# 1 << 3 == "0...01000"
# 1 << 5 ^ 1 << 3 == "0...0101000"
binaryString ^= 1 << 3
print("{0:032b}".format(binaryString))


00000000000000000000000000000001
00000000000000000000000000100000
00000000000000000000000000101000


### Exclusive Ors (XORs) for Flipping Bits 

Exclusive or, XOR for short, is a logical operation which evaluates to true if only one of the arguments is true. I.e., 1 ^ 0 == 0 ^ 1 == 1 and 0 ^ 0 == 1 ^ 1 == 0 (*note: in most languages XOR is called by using ^ or ^= if we want to apply it and store it to the same string*).

Because we use XOR, before flipping a bit on we must check that it has not already been initialized. If we do not check this, we risk uninitializing ticks that should remain initialized. This can be done dynamically when updating the state to increment liquidity. For now let's assume that this check was already done.

In [774]:
# 1 == "0...001"
binaryString = 1
print("{0:032b}".format(binaryString))

#1 << 5 == "0...0100000"
binaryString = binaryString << 5
print("{0:032b}".format(binaryString))

#Turning a bit on at index 3
# 1 << 3 == "0...01000"
# 1 << 5 ^= 1 << 3 == "0...0101000"
binaryString ^= 1 << 3
print("{0:032b}".format(binaryString))

# Turning a bit off at index 3
# binaryString ^= 1 << 3 == "0...0100000"
binaryString ^= 1 << 3
print("{0:032b}".format(binaryString))


00000000000000000000000000000001
00000000000000000000000000100000
00000000000000000000000000101000
00000000000000000000000000100000


### Putting it all Together

Putting this all together in a single function lets us retrieve which tick we are adding/removing liquidity to and flip the bit corresponding to the index when necessary (*note: since maps are mutable in python, it is in essence "passed by reference" so we can edit it in a function - excuse my bad programming, should've made it a class :/*)

In [775]:
def flipTick(bitmap : dict, tickIndex : int):
    wordIndex, localTickIndex = position(tickIndex)
    mask = 1 << localTickIndex
    bitmap[wordIndex] ^= mask

Let's see this in action.

In [776]:
print("\n --- BEFORE FIRST FLIP ---")
print("{0:032b}".format(bitmap[-2]),
    "{0:032b}".format(bitmap[-1]), 
    "{0:032b}".format(bitmap[0]), 
    "{0:032b}".format(bitmap[1]), 
    "{0:032b}".format(bitmap[2]))

flipTick(bitmap, -33)
flipTick(bitmap, -5)
flipTick(bitmap, 1)
flipTick(bitmap, 3)
flipTick(bitmap, 21)
flipTick(bitmap, 61)
flipTick(bitmap, 66)

print("\n --- AFTER FIRST FLIP ---")
print("{0:032b}".format(bitmap[-2]),
    "{0:032b}".format(bitmap[-1]), 
    "{0:032b}".format(bitmap[0]), 
    "{0:032b}".format(bitmap[1]), 
    "{0:032b}".format(bitmap[2]))

flipTick(bitmap, -33)
flipTick(bitmap, -5)
flipTick(bitmap, 1)
flipTick(bitmap, 3)
flipTick(bitmap, 21)
flipTick(bitmap, 61)
flipTick(bitmap, 66)

print("\n --- AFTER SECOND FLIP ---")
print("{0:032b}".format(bitmap[-2]),
    "{0:032b}".format(bitmap[-1]), 
    "{0:032b}".format(bitmap[0]), 
    "{0:032b}".format(bitmap[1]), 
    "{0:032b}".format(bitmap[2]))

# Reflipping to initialize map for functions that come later 
flipTick(bitmap, -33)
flipTick(bitmap, -5)
flipTick(bitmap, 1)
flipTick(bitmap, 3)
flipTick(bitmap, 21)
flipTick(bitmap, 61)
flipTick(bitmap, 66)


 --- BEFORE FIRST FLIP ---
00000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000

 --- AFTER FIRST FLIP ---
10000000000000000000000000000000 00001000000000000000000000000000 00000000001000000000000000001010 00100000000000000000000000000000 00000000000000000000000000000100

 --- AFTER SECOND FLIP ---
00000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000 00000000000000000000000000000000


## Finding the Next Tick

### Definitions

**Current Tick**: The tick with the best priced liquidity for a given swap direction.

**Direction**: Whether a trade is token 0 for token 1 or token 1 for token 0

**Next Tick**: The tick with the next best priced liquidity for a given a swap direction. After we deplete the liquidity at the current tick, we need to efficiently find the next tick to swap through.

### Finding Next Tick for Swaps 0 to 1

We start by looking at a case where the next tick is within the same word. Let's say we are starting at the tick indexed 3 in the word indexed by 0. Based on previous values set, the next tick index should be 21.

To find the next index within a word, we first have to construct a mask which sets every bit with an index greater than the index of the local tick index to 1. 

An efficient way to do this is to first bitshift a 1 into the index of the local tick index. By subtracting 1 from this number we make it so every bit before the local index is now a 1 (*note: for a lesson on binary addition and subtraction check out these __[lecture slides](https://faculty-web.msoe.edu/johnsontimoj/Common/FILES/binary_addition_subtraction.pdf)__*). Finally we bitwise negate the mask to so to make sure every bit with an index greater than or equal to the local index is a 1, and everything before it is a 0 (*note: we have to add (1 << 32) since there is no unsigned integer type in python. By bitshifting left a 1 32 places, we get the binary of unsigned integer post-negation. *)

In [777]:
currentTick = 3
wordIndex, localTickIndex = position(currentTick + 1)

print("\nmask = 1 << localTickIndex")
mask = 1 << localTickIndex
print("{0:032b}".format(mask))

print("\nmask -= 1")
mask -= 1 
print("{0:032b}".format(mask))

# We need to add 1 << 32 since python doesn't have unsigned integers, 
# but normally just using ~ would be enough.
print("\nmask = ~(mask)  + (1 << 32)")
mask = ~(mask)  + (1 << 32)
print("{0:032b}".format(mask))

# In a single line it looks as follows:
print("\nmask = ~((1 << localTickIndex) - 1) + (1 << 32)")
mask = ~((1 << localTickIndex) - 1) + (1 << 32)
print("{0:032b}".format(mask))

print("\nbitmap[wordIndex]")
print("{0:032b}".format(bitmap[wordIndex]))


mask = 1 << localTickIndex
00000000000000000000000000010000

mask -= 1
00000000000000000000000000001111

mask = ~(mask)  + (1 << 32)
11111111111111111111111111110000

mask = ~((1 << localTickIndex) - 1) + (1 << 32)
11111111111111111111111111110000

bitmap[wordIndex]
00000000001000000000000000001010


We then call do a bitwise AND on the word and the mask to store a string where all of the bits that correspond to ticks **with liquidity** and with indices greater than or equal to the local index are 1. 

In [778]:
print("\nmasked = bitmap[wordIndex] & mask")
masked = bitmap[wordIndex] & mask
print("{0:032b}".format(masked))


masked = bitmap[wordIndex] & mask
00000000001000000000000000000000


Before proceeding with anything else we want to make sure that there are actually ticks left to swap through that are indexed within the current word. A convenient way to do this is to just compare it to 0 and see if they're not the same. If there aren't we want to hop to the next word and check there.  

In [779]:
initialized = masked != 0
# Will evaluate to True since we can see that there are bits indexed by 3 and 21.
print(initialized)

True


Least significant bit is a way to find the next (larger) initialized tick. It requires that the input x is not 0 since we should just skip to the next word in that case.

The function relies on a parameter for the next index r being decremented sufficiently. It ensures this by checking whether the masked word shares any bits with a max uint (*Note: (1 << y) - 1 is the max uint of y*) and either bit shifting right x by the number of bits of the current max uint ("y" from previous note), or subtracting from the number of bits from the index.

This can be confusing at first but one way to understand what's going on is:
- We can at most bitshift right 31 places (31 = 16 + 8 + 4 + 2 + 1), 
- When the next tick (lowest index higher than current) in the masked word x is at an index greater than 15, we know there are at least 16 bits of space until we reach. 
- By bitshifting right 16 places in this case and not subtracting 16 from r, we are saying that 16 < r <= 31 since we can now only decrement r by 15 at maximum (15 = 8 + 4 + 2 + 1)
- If the next tick (lowest index higher than current) in the masked word x had been at an index less than or equal to 15 then we would have subtracted 16 from r (r = 31 - 16 = 15) telling us that r <= 15
- Repeat these steps for 8, 4, 2, 1

*Note this is basically a binary search for finding the next tick number*

In [780]:
def leastSignificantBit(x : int) -> int: 
    assert x != 0

    r = 31
    #print("\n")
    #print("x:", "{0:032b}".format(x))
    #print("r:", r)
    #print("((1 << 16) - 1):", "{0:032b}".format(((1 << 16) - 1)))
    if (x & ((1 << 16) - 1) > 0):
        r -= 16
    else:
        x >>= 16
    #print("\n")
    #print("x:", "{0:032b}".format(x))
    #print("r:", r)
    #print("((1 << 8) - 1):", "{0:032b}".format(((1 << 8) - 1)))
    if (x & ((1 << 8) - 1) > 0):
        r -= 8
    else:
        x >>= 8
    #print("\n")
    #print("x:", "{0:032b}".format(x))
    #print("r:", r)
    #print("((1 << 4) - 1):", "{0:032b}".format(((1 << 4) - 1)))
    if (x & ((1 << 4) - 1) > 0):
        r -= 4
    else:
        x >>= 4
    #print("\n")
    #print("x:", "{0:032b}".format(x))
    #print("r:", r)
    if (x & ((1 << 2) - 1) > 0):
        r -= 2
    else:
        x >>= 2
    #print("\n")
    #print("x:", "{0:032b}".format(x))
    #print("r:", r)
    if (x & 1 > 0):
        r -= 1
    return r

Some potentially helpful print statements to understand what's going on.

In [781]:
#print(leastSignificantBit(masked))
#print("{0:032b}".format(masked))
#print(currentTick + 1 + leastSignificantBit(masked) - localTickIndex)
#print(2 ** 32 - 1)
#print(localTickIndex)
#print("{0:032b}".format((1 << 32) - 1))

If initialized is false, there are no ticks left in this direction so we signal that the next tick to check is going to be the first bit in the following word. When we run the code, we see that the correct next tick was found (21)!

In [782]:
if initialized:
    next = currentTick + 1 + (leastSignificantBit(masked) - localTickIndex)
else:
    next = currentTick + 1 + 32 - localTickIndex

print(next)

21


Automating the rest of the tick search until there are no more initialized ticks. Notice that when initialized == false, meaning that there are no more ticks to check in this word, the next ticks are 32 or 64 which are also the first ticks in a new word.

In [783]:

for i in range(4):
    currentTick = next
    wordIndex, localTickIndex = position(currentTick + 1)

    mask = ~((1 << localTickIndex) - 1) + (1 << 32)
    masked = bitmap[wordIndex] & mask

    initialized = masked != 0

    if initialized:
        next = currentTick + 1 + (leastSignificantBit(masked) - localTickIndex)
    else:
        next = currentTick + 1 + 32 - localTickIndex
    
    print("\nAre there ticks left: ", initialized)
    print("Next tick: ", next)

currentTick = next



Are there ticks left:  False
Next tick:  32

Are there ticks left:  True
Next tick:  61

Are there ticks left:  False
Next tick:  64

Are there ticks left:  True
Next tick:  66


### Finding Next Tick for Swaps 1 to 0

The current tick is 66 and our goal is to efficiently iterate back to the first tick (at index 1).

To find the next index within a word, we first have to construct a mask which sets every bit with an index less than or equal to the index of the local tick index to 1. 

An efficient way to do this is to first bitshift a 1 into the index of the local tick index. By subtracting 1 from this number we make it so every bit before the local index is now a 1. Finally we add rightly shifted 1 over the the same number of times as the local tick index. This is to make the index corresponding to the local tick index also 1 for the mask.

In [784]:
wordIndex, localTickIndex = position(currentTick)

print("\nmask = 1 << localTickIndex")
mask = 1 << localTickIndex
print("{0:032b}".format(mask))

print("\nmask -= 1")
mask -= 1 
print("{0:032b}".format(mask))

print("\nmask += 1 << localTickIndex")
mask += 1 << localTickIndex
print("{0:032b}".format(mask))

# In a single line it looks as follows:
print("\nmask = ~((1 << localTickIndex) - 1) + (1 << 32)")
mask = (1 << localTickIndex) - 1 + (1 << localTickIndex)
print("{0:032b}".format(mask))

print("\nbitmap[wordIndex]")
print("{0:032b}".format(bitmap[wordIndex]))


mask = 1 << localTickIndex
00000000000000000000000000000100

mask -= 1
00000000000000000000000000000011

mask += 1 << localTickIndex
00000000000000000000000000000111

mask = ~((1 << localTickIndex) - 1) + (1 << 32)
00000000000000000000000000000111

bitmap[wordIndex]
00000000000000000000000000000100


We create the masked version of the word the same as for the other direction (0 to 1) and evaluate whether there are any ticks left below the current tick.

In [785]:
print("\nmasked = bitmap[wordIndex] & mask")
masked = bitmap[wordIndex] & mask
print("{0:032b}".format(masked))

initialized = masked != 0

# Will evaluate to False since there are not bits >62 and <66
print(initialized)


masked = bitmap[wordIndex] & mask
00000000000000000000000000000100
True


Most significant bit is a way to find the next (smaller) initialized tick. It requires that the input x is not 0 since we should just skip to the next word in that case.

The function relies on a parameter for the next index r being incremented sufficiently. It ensures this by iteratively checking if the masked word is greater than some number which helps tell us something about the next tick's bit index.

This can be confusing, but one way to understand what's going on is:
- We can at most bitshift right 31 places (31 = 16 + 8 + 4 + 2 + 1), 
- When the next tick (highest index lower than current) in the masked word x is at an index greater than or equal to 16, we know that x >= 2 ** 16 = 1 << 16. 
- By bitshifting right 16 places in this case and adding 16 to r, we are saying that 16 <= r <= 31 since we can now only increment r by 15 at maximum (15 = 8 + 4 + 2 + 1)
- If the next tick (highest index lower than current) in the masked word x had been at an index less than 16 then we would have done nothing (r = 10 still) telling us that r <= 15 since we can max increment r by 15.
- Repeat for 8, 4, 2, 1

*Note this is basically a binary search for finding the next tick number*

In [786]:
def mostSignificantBit(x : int) -> int: 
    assert x != 0

    r = 0
    #print("\n")
    #print("x:", "{0:032b}".format(x))
    #print("r:", r)
    #print("1 << 16:", "{0:032b}".format((1 << 16)))
    if (x >= (1 << 16)):
        r += 16
        x >>= 16
    #print("\n")
    #print("x:", "{0:032b}".format(x))
    #print("r:", r)
    #print("1 << 8:", "{0:032b}".format(1 << 8))
    if (x >= (1 << 8)):
        r += 8
        x >>= 8
    #print("\n")
    #print("x:", "{0:032b}".format(x))
    #print("r:", r)
    #print("1 << 4:", "{0:032b}".format(1 << 4))
    if (x >= (1 << 4)):
        r += 4
        x >>= 4
    #print("\n")
    #print("x:", "{0:032b}".format(x))
    #print("r:", r)
    if (x >= 4):
        r += 2
        x >>= 2
    #print("\n")
    #print("x:", "{0:032b}".format(x))
    #print("r:", r)
    if (x >= 2):
        r += 1
    return r

If initialized is false, there are no ticks left in this direction so we signal that the next tick to check is going to be the first bit in the following word. When we run the code, we see that the correct next tick was found (64) since there were no more ticks less than 66 and within this word!

In [787]:
if initialized:
    print("check")
    next = currentTick - localTickIndex - mostSignificantBit(masked)
else:
    print("check2")
    next = currentTick - localTickIndex

print(next)

check
62


### Final Next Tick Function

Putting it all together...

In [788]:
def nextInitTickWithinWord(bitmap : dict, 
                        currentTick : int,
                        inputToken : int ) -> Tuple[int, bool]:
 
    # Next tick calculations for trades from token 1 to 0
    if inputToken:
        None
    # Next tick calculations for trades from token 0 to 1
    else:
        wordIndex, localTickIndex = position(currentTick + 1)

        mask = ~((1 << localTickIndex) - 1) + (1 << 32)
        masked = bitmap[wordIndex] & mask
        
        initialized = masked != 0

        if initialized:
            next = currentTick + 1 + (leastSignificantBit(masked) - localTickIndex)
        else:
            next = currentTick + 1 + (1 << 32) - 1 - localTickIndex

    return (next )

Hopefully this has been helpful :)